In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11b5cd908>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x120a63fd0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x120a692e8>)

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [21]:
sess.run(tf.global_variables_initializer())

In [13]:
# [None, 784] x [784, 10] = [None, 10]
y = tf.matmul(x, W) + b

In [14]:
y

<tf.Tensor 'add_2:0' shape=(?, 10) dtype=float32>

In [16]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [17]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [33]:
train_step

<tf.Operation 'GradientDescent' type=NoOp>

In [22]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    # sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})

In [26]:
y, tf.argmax(y, axis=0), tf.argmax(y, axis=1)

(<tf.Tensor 'add_2:0' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'ArgMax_2:0' shape=(10,) dtype=int64>,
 <tf.Tensor 'ArgMax_3:0' shape=(?,) dtype=int64>)

In [27]:
correct_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_, axis=1))

In [28]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [31]:
accuracy

<tf.Tensor 'Mean_2:0' shape=() dtype=float32>

In [34]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9175
0.9175


## CNN

In [43]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [46]:
a = tf.truncated_normal((3, 3), stddev=0.1)
print(a.eval())
b = tf.constant(0.1, shape=(3, 1))
print(b.eval())

[[-0.05010616 -0.04017609 -0.00569539]
 [ 0.13421093 -0.00210931 -0.08559476]
 [ 0.00119102 -0.03151929 -0.04067332]]
[[ 0.1]
 [ 0.1]
 [ 0.1]]


In [47]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [48]:
W_conv1 = weight_variable([5, 5, 1, 32])  # h, w, in_channel, out_channel
b_conv1 = bias_variable([32])

In [49]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>

In [50]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [52]:
x_image, W_conv1

(<tf.Tensor 'Reshape_6:0' shape=(?, 28, 28, 1) dtype=float32>,
 <tf.Variable 'Variable_2:0' shape=(5, 5, 1, 32) dtype=float32_ref>)

In [53]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [55]:
h_conv1, h_pool1

(<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>,
 <tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>)

In [56]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [57]:
h_conv2, h_pool2

(<tf.Tensor 'Relu_1:0' shape=(?, 14, 14, 64) dtype=float32>,
 <tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>)

In [58]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [60]:
h_pool2_flat, h_fc1

(<tf.Tensor 'Reshape_7:0' shape=(?, 3136) dtype=float32>,
 <tf.Tensor 'Relu_2:0' shape=(?, 1024) dtype=float32>)

In [61]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [63]:
h_fc1_drop

<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float32>

In [64]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [65]:
y_conv

<tf.Tensor 'add_6:0' shape=(?, 10) dtype=float32>

In [66]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test_accuracy %g' % accuracy.eval(feed_dict={
                x: mnist.test_images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08
step 100, training accuracy 0.96
step 200, training accuracy 0.94
step 300, training accuracy 0.92
step 400, training accuracy 0.88
step 500, training accuracy 0.88
step 600, training accuracy 0.96
step 700, training accuracy 0.88
step 800, training accuracy 1
step 900, training accuracy 0.98
step 1000, training accuracy 0.98


KeyboardInterrupt: 